In [ ]:
# Uncomment if notebook is run in Google Colab
# %%capture
# !pip install datasets

In [70]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re

from evaluate import load

In [2]:
# Helper function
def clean_string(string):
    """
    Cleans the input string by removing asterisks, converting to lowercase,
    and removing trailing whitespace characters.
    """
    
    string = re.sub('\*', '', string).lower().rstrip()
    return string

In [3]:
webis_tldr = load_dataset('reddit', split='train')
reddit_tifu = load_dataset('reddit_tifu', 'long', split='train')

Found cached dataset reddit (C:/Users/Anna/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e)
Found cached dataset reddit_tifu (C:/Users/Anna/.cache/huggingface/datasets/reddit_tifu/long/1.1.0/3136b11fbef3f2517de1d720621af110bd29e6083aebeab0d8ec198c9f95dc95)


In [33]:
webis_tldr_df = webis_tldr.to_pandas()
reddit_tifu_df = reddit_tifu.to_pandas()

In [34]:
webis_tldr_tifu_df = webis_tldr_df.loc[webis_tldr_df['subreddit'] == "tifu"]

In [35]:
len(webis_tldr_tifu_df)

52219

In [36]:
webis_tldr_tifu_df.head()

,author,body,normalizedBody,subreddit,subreddit_id,id,content,summary
471,Leterren,"Harken all to my tale of dream pissing, for I ...","Harken all to my tale of dream pissing, for I ...",tifu,t5_2to41,c7f5mib,"Harken all to my tale of dream pissing, for I ...","For those keeping count, I had a dream-within-..."
530,TIFU_mod,[Link](\n\n**TIFU by combining magnets with my...,[Link]( \n TIFU by combining magnets with my u...,tifu,t5_2to41,c7koxqi,Link]( \n TIFU by combining magnets with my ur...,I'm not sure I'd advise sticking 74 magnets up...
531,TIFU_mod,[Link](\n\n**TIFU by falling asleep while watc...,[Link]( \n TIFU by falling asleep while watchi...,tifu,t5_2to41,c7koxw9,Link]( \n TIFU by falling asleep while watchin...,I fell asleep after masturbating to porn and m...
532,TIFU_mod,"[Link](\n\n**TIFU. Dutch-ovened myself, puked....","[Link]( \n TIFU. Dutch-ovened myself, puked. \...",tifu,t5_2to41,c7koy14,"Link]( \n TIFU. Dutch-ovened myself, puked. \n...","Dutch ovened myself, barfed. I'm also probably..."
533,TIFU_mod,[Link](\n\n**TIFU while peeing.**\n\nJust happ...,[Link]( \n TIFU while peeing. \n Just happened...,tifu,t5_2to41,c7koycb,Link]( \n TIFU while peeing. \n Just happened....,"Thumb slipped, junk whipped, everything dripped."


In [37]:
reddit_tifu_df.head()

,ups,num_comments,upvote_ratio,score,documents,tldr,title
0,115.0,23.0,0.88,115.0,this actually happened a couple of years ago. ...,confuse a 5th grade girl for a boy in front of...,gender-stereotyping
1,16.0,12.0,0.79,16.0,"it was last october, but i'm feeling the fall-...","i found my estranged dad, thought i loved him ...",telling my dad that i love him.
2,55.0,10.0,0.85,55.0,so i had the brilliant idea to use veet hair r...,had my balls burned by sauron and was left dev...,i was deveeted...
3,90.0,20.0,0.92,90.0,today i was going to have a bath after a long ...,peppermint + bath = burning cold ladybits.,wanting a pepperminty bath.
4,81.0,18.0,0.79,81.0,"i haven't had a bath in practically years so, ...","got too high and too hot in the bath, almost c...",having a spliff in the bath.


In [38]:
webis_source_texts = webis_tldr_tifu_df['content']
tifu_source_texts = reddit_tifu_df['documents']

In [39]:
webis_source_texts_df = pd.DataFrame({'index': webis_source_texts.index, 'text': webis_source_texts.values})
tifu_source_texts_df = pd.DataFrame({'index': tifu_source_texts.index, 'text': tifu_source_texts.values})

In [40]:
webis_source_texts_df['text_uncased'] = webis_source_texts_df['text'].str.lower()
tifu_source_texts_df['text_uncased'] = tifu_source_texts_df['text'].str.lower()

In [41]:
webis_source_texts_df['text_clean'] = webis_source_texts_df['text_uncased'].apply(clean_string)
tifu_source_texts_df['text_clean'] = tifu_source_texts_df['text_uncased'].apply(clean_string)

In [42]:
webis_source_texts_list = webis_source_texts_df['text_clean'].tolist()
tifu_source_texts_list = tifu_source_texts_df['text_clean'].tolist()

In [43]:
webis_source_texts_df.head()

,index,text,text_uncased,text_clean
0,471,"Harken all to my tale of dream pissing, for I ...","harken all to my tale of dream pissing, for i ...","harken all to my tale of dream pissing, for i ..."
1,530,Link]( \n TIFU by combining magnets with my ur...,link]( \n tifu by combining magnets with my ur...,link]( \n tifu by combining magnets with my ur...
2,531,Link]( \n TIFU by falling asleep while watchin...,link]( \n tifu by falling asleep while watchin...,link]( \n tifu by falling asleep while watchin...
3,532,"Link]( \n TIFU. Dutch-ovened myself, puked. \n...","link]( \n tifu. dutch-ovened myself, puked. \n...","link]( \n tifu. dutch-ovened myself, puked. \n..."
4,533,Link]( \n TIFU while peeing. \n Just happened....,link]( \n tifu while peeing. \n just happened....,link]( \n tifu while peeing. \n just happened....


In [45]:
webis_source_texts_df.rename(columns={'index': 'webis_index'}, inplace=True)
webis_source_texts_df.drop(columns=["text", "text_uncased"], inplace=True)

In [53]:
webis_source_texts_df.head()

,webis_index,text_clean
0,471,"harken all to my tale of dream pissing, for i ..."
1,530,link]( \n tifu by combining magnets with my ur...
2,531,link]( \n tifu by falling asleep while watchin...
3,532,"link]( \n tifu. dutch-ovened myself, puked. \n..."
4,533,link]( \n tifu while peeing. \n just happened....


In [44]:
tifu_source_texts_df.head()

,index,text,text_uncased,text_clean
0,0,this actually happened a couple of years ago. ...,this actually happened a couple of years ago. ...,this actually happened a couple of years ago. ...
1,1,"it was last october, but i'm feeling the fall-...","it was last october, but i'm feeling the fall-...","it was last october, but i'm feeling the fall-..."
2,2,so i had the brilliant idea to use veet hair r...,so i had the brilliant idea to use veet hair r...,so i had the brilliant idea to use veet hair r...
3,3,today i was going to have a bath after a long ...,today i was going to have a bath after a long ...,today i was going to have a bath after a long ...
4,4,"i haven't had a bath in practically years so, ...","i haven't had a bath in practically years so, ...","i haven't had a bath in practically years so, ..."


In [47]:
tifu_source_texts_df.rename(columns={'index': 'tifu_index'}, inplace=True)
tifu_source_texts_df.drop(columns=["text", "text_uncased"], inplace=True)

In [50]:
tifu_source_texts_df

,tifu_index,text_clean
0,0,this actually happened a couple of years ago. ...
1,1,"it was last october, but i'm feeling the fall-..."
2,2,so i had the brilliant idea to use veet hair r...
3,3,today i was going to have a bath after a long ...
4,4,"i haven't had a bath in practically years so, ..."
...,...,...
42134,42134,this happened back in middle school.\n\nmy fam...
42135,42135,my girlfriend told me she has no hair beneath ...
42136,42136,today at work i accidentally crashed a row of ...
42137,42137,so as u can tell from the title it didn't happ...


## Finding Reddit TIFU elements in Webis-TLDR-17

In [54]:
# Broadcasting to compare elements element-wise
comparison_matrix = webis_source_texts_df['text_clean'].values[:, None] == tifu_source_texts_df['text_clean'].values

In [56]:
len(webis_source_texts_df)

52219

In [55]:
comparison_matrix

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [81]:
comparison_matrix.shape

(52219, 42139)

In [61]:
# Get the values of the comparison matrix that are True
matched_values = webis_source_texts_df['text_clean'][comparison_matrix.any(axis=1)].tolist()

In [80]:
len(matched_values)

5735

In [71]:
# Get the coordinates (indices) of True values in the comparison matrix
indices = np.where(comparison_matrix)

In [77]:
# Convert the coordinates to a list of tuples
coordinates = list(zip(indices[0], indices[1]))

In [83]:
len(coordinates)

5745

In [84]:
coordinates[:10]

[(3881, 4093),
 (3890, 4083),
 (3891, 4082),
 (3894, 11560),
 (3899, 11557),
 (3906, 11551),
 (3909, 11548),
 (3914, 11543),
 (3933, 11528),
 (3939, 11523)]

## Finding Webis-TLDR-17 elements in Reddit TIFU

In [64]:
# Broadcasting to compare elements element-wise
comparison_matrix_2 = tifu_source_texts_df['text_clean'].values[:, None] == webis_source_texts_df['text_clean'].values

In [66]:
# Get the values of the comparison matrix that are True
matched_values_2 = tifu_source_texts_df['text_clean'][comparison_matrix_2.any(axis=1)].tolist()

In [67]:
len(matched_values_2)

5712

In [94]:
# Get the coordinates (indices) of True values in the comparison matrix
indices_2 = np.where(comparison_matrix_2)

In [95]:
# Convert the coordinates to a list of tuples
coordinates_2 = list(zip(indices_2[0], indices_2[1]))

In [97]:
len(coordinates_2)

5745